In [ ]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch import nn as nn
from torch.nn import init as init
import pandas as pd


print(torch.__version__)

# Training the Model

In [ ]:
class BinaryModel(nn.Module):
    def __init__(self):
        super(BinaryModel, self).__init__()
        self.layer1 = nn.Linear(9,18)
        self.layer2 = nn.Linear(18,12)
        self.layer3 = nn.Linear(12,6)
        self.layer4 = nn.Linear(6,1)
        self.sigmoid = nn.sigmoid()

    def forward(self,x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return self.sigmoid(x)

model = BinaryModel()

In [ ]:
for layer in mode.modules():
    if isinstance(layer, nn.Linear):
        init.xavier_uniform_(layer.weight)
        init.zeros_(layer.bias)

## Prepping Data

In [ ]:
#reads from the csv file
df = pd.read_csv("./water_potability.csv")
#fills in null values within the data frame with the mean of the appropriate column
df.fillna(df.mean(),inplace=True)
df.isnull().sum()

In [ ]:
#Features What we're feeding
features = df.drop("Potability", axis=1)
#Label is the "truth", What we're looking for
labels = df['Potability']

In [ ]:
### standardize and normalize Features

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
standardized_features = scaler.fit_transform(features)
standardized_features[:3]

In [ ]:
train_test_split(standardized_features, labels, test_size=0.8, random_state=42)

In [ ]:
training_f, testing_f, training_l, testing_l = train_test_split(standardize_features, labels, test_size=0.2)

In [ ]:
training_f = torch.tensor(training_f, dtype=torch.float32)
training_l = torch.tensor(training_l.values, dtype=torch.float32).view(-1,1)
testing_f = torch.tensor(testing_f, dtype=torch.float32)
testing_l = torch.tensor(testing_l.values, dtype=torch.float32).view(-1,1)
# testing_labels

## Creating Data Loaders

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(training_f, training_l)
testing_dataset = TensorDataset(testing_f, testing_l)
training_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
testing_loader = DataLoader(testing_dataset, batch_size=32, shuffle=True)
testing_loader

## Training Loop

In [ ]:
from torch import optim

optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [ ]:
num_of_epochs = 200

for epoch in range(num_of_epochs):
  model.train()
  for features, labels in training_loader:
    optimizer.zero_grad()
    output = model(features.float())
    loss = criterion(output, labels.view(-1, 1).float())
    loss.backward()
    optimizer.step()

  print(f"Epoch {epoch+1}/{num_of_epochs}, Loss: {loss.item()}")

In [ ]:
from torchmetrics import Accuracy

accuracy = Accuracy(task='binary')

model.eval()
with torch.no_grad():
  for features, labels in testing_loader:
    output = model(features)
    predicted = output.round()
    accuracy.update(predicted, labels)

print(f"Accuracy: {accuracy.compute().item()}")